**Table of contents**<a id='toc0_'></a>    
- [Sender receiver operations](#toc1_)    
  - [Sender receiver fixed effects](#toc1_1_)    
  - [Sender receiver random effects](#toc1_2_)    
  - [R equivalent](#toc1_3_)    
  - [SR to dyadic format](#toc1_4_)    
- [Dyadic effect](#toc2_)    
  - [Dyadic fixed effects](#toc2_1_)    
  - [Dyadic random effects](#toc2_2_)    
  - [R equivalent](#toc2_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
%load_ext rpy2.ipython

Since the seed is not modified, all sampling remains the same. We will create the parameters using the BI package and perform the mathematical operations in R to assess where the BI mathematical operations might be failing.

# <a id='toc1_'></a>[Sender receiver operations](#toc0_)

In [2]:
from main import*
N_id = 3
sample = True
focal_individual_predictors = jnp.array([1, 2, 3]).reshape(1,3)
target_individual_predictors = jnp.array([1, 2, 3]).reshape(1,3)
focal_effects = dist.normal(0, 1, shape=(1,), sample = sample, name = 'focal_effects')
target_effects =  dist.normal( 0, 1, shape= (1,), sample = sample, name = 'target_effects')
ff = jnp.stack([focal_effects @ focal_individual_predictors, target_effects @  target_individual_predictors], axis = -1)

sr_raw =  dist.normal(0, 1, shape=(2, N_id), name = 'sr_raw', sample = sample)
sr_sigma =  dist.exponential( 1, shape= (2,), name = 'sr_sigma', sample = sample)
sr_L = dist.lkjcholesky(2, 2, name = "sr_L", sample = sample)
rf = numpyro.deterministic('sr_rf',(((sr_L @ sr_raw).T * sr_sigma)))

print("sr_ff--------------------------------")
print(ff)
print("sr_rf--------------------------------")
print(rf)
print("sr_ff + sr_rf--------------------------------")
print(rf + ff)


sr_ff--------------------------------
[[-0.20584226 -0.20584226]
 [-0.4116845  -0.4116845 ]
 [-0.61752677 -0.61752677]]
sr_rf--------------------------------
[[ 0.04578188  1.9610099 ]
 [-0.3127802   0.7283852 ]
 [ 0.15827817 -0.35851383]]
sr_ff + sr_rf--------------------------------
[[-0.16006038  1.7551676 ]
 [-0.7244647   0.3167007 ]
 [-0.4592486  -0.9760406 ]]


In [3]:
sr_terms, focal_effects, target_effects = bi.net.nodes_terms(focal_individual_predictors, target_individual_predictors, sample = True)
sr_rf, sr_raw, sr_sigma, sr_L = bi.net.nodes_random_effects(N_id, cholesky_density = 2, sample = True)

print("sr_ff--------------------------------")
print(sr_terms)
print("sr_rf--------------------------------")
print(sr_rf)
print("sr_ff + sr_rf--------------------------------")
print(sr_terms + sr_rf)


sr_ff--------------------------------
[[-0.20584226 -0.20584226]
 [-0.4116845  -0.4116845 ]
 [-0.61752677 -0.61752677]]
sr_rf--------------------------------
[[ 0.04578188  1.9610099 ]
 [-0.3127802   0.7283852 ]
 [ 0.15827817 -0.35851383]]
sr_ff + sr_rf--------------------------------
[[-0.16006038  1.7551676 ]
 [-0.7244647   0.3167007 ]
 [-0.4592486  -0.9760406 ]]


## <a id='toc1_1_'></a>[Sender receiver fixed effects](#toc0_)

In [4]:
from main import *
import numpy as np
N_id = 3
focal_individual_predictors = jnp.array([1, 2, 3]).reshape(1,3)
target_individual_predictors = jnp.array([1, 2, 3]).reshape(1,3)
sr_ff, focal_effects, target_effects = bi.net.nodes_terms(focal_individual_predictors, target_individual_predictors, sample=True, diag = True)
focal_effects = np.array(focal_effects)
target_effects = np.array(target_effects)
focal_individual_predictors = np.array(focal_individual_predictors)
target_individual_predictors = np.array(target_individual_predictors)
sr_ff = np.array(sr_ff)

focal_effects--------------------------------------------------------------------------------
[-0.20584226]
target_effects--------------------------------------------------------------------------------
[-0.20584226]
terms--------------------------------------------------------------------------------
[[-0.20584226 -0.20584226]
 [-0.4116845  -0.4116845 ]
 [-0.61752677 -0.61752677]]


In [5]:
%%R -i focal_individual_predictors 
focal_individual_predictors

     [,1] [,2] [,3]
[1,]    1    2    3


In addition: Warning message:
In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages


In [6]:
%%R -i target_individual_predictors

target_individual_predictors
N_id = length(focal_individual_predictors)

In [7]:
%%R -i focal_effects  
focal_effects

[1] -0.2058423


In [8]:
%%R -i target_effects 

target_effects

[1] -0.2058423


In [9]:
%%R -i sr_ff
sr_ff

           [,1]       [,2]
[1,] -0.2058423 -0.2058423
[2,] -0.4116845 -0.4116845
[3,] -0.6175268 -0.6175268


## <a id='toc1_2_'></a>[Sender receiver random effects](#toc0_)

In [10]:
sr_rf, sr_raw, sr_sigma, sr_L = bi.net.nodes_random_effects(N_id, cholesky_density = 2, sample=True, diag = True) 
sr_rf = np.array(sr_rf)
sr_raw = np.array(sr_raw)
sr_sigma = np.array(sr_sigma)
sr_L  = np.array(sr_L)

sr_raw--------------------------------------------------------------------------------
[[ 0.18784384 -1.2833426   0.6494181 ]
 [ 1.2490593   0.24447003 -0.11744965]]
sr_sigma--------------------------------------------------------------------------------
[0.24372306 1.630274  ]
sr_L--------------------------------------------------------------------------------
[[ 1.          0.        ]
 [-0.16010584  0.9870998 ]]
rf--------------------------------------------------------------------------------
[[ 0.04578188  1.9610099 ]
 [-0.3127802   0.7283852 ]
 [ 0.15827817 -0.35851383]]


In [11]:
%%R -i sr_raw
sr_raw = t(sr_raw)
sr_raw

           [,1]       [,2]
[1,]  0.1878438  1.2490593
[2,] -1.2833426  0.2444700
[3,]  0.6494181 -0.1174496


In [12]:
%%R -i sr_rf
sr_rf

            [,1]       [,2]
[1,]  0.04578188  1.9610099
[2,] -0.31278020  0.7283852
[3,]  0.15827817 -0.3585138


In [13]:
%%R -i sr_sigma
sr_sigma

[1] 0.2437231 1.6302741


In [14]:
%%R -i sr_L
sr_L

           [,1]      [,2]
[1,]  1.0000000 0.0000000
[2,] -0.1601058 0.9870998


## <a id='toc1_3_'></a>[R equivalent](#toc0_)

In [15]:
%%R
sr_termsR = NULL
sr_rfR = NULL
srR = NULL
for (i in 1:N_id) {
  tmp <- numeric(2)  # Initialize a vector of size 2

  # Compute the dot product for the focal and target predictors
  tmp[1] <- sum(focal_effects %*% focal_individual_predictors[i])
  tmp[2] <- sum(target_effects %*% target_individual_predictors[i])
  
  sr_termsR = rbind(sr_termsR, tmp)

  pre_multiply_result <- diag(sr_sigma) %*% sr_L
  sr_rfR = rbind(sr_rfR, t(pre_multiply_result %*% sr_raw[i,]) )

  srR[[i]] <- t(pre_multiply_result %*% sr_raw[i,]) + tmp
}
print('sr_ff--------------------------------')
print(sr_termsR)
print('sr_rf --------------------------------')
print(sr_rfR)
print('sr_ff + sr_rf--------------------------------')
print(do.call('rbind',srR))


[1] "sr_ff--------------------------------"
          [,1]       [,2]
tmp -0.2058423 -0.2058423
tmp -0.4116845 -0.4116845
tmp -0.6175268 -0.6175268
[1] "sr_rf --------------------------------"
            [,1]       [,2]
[1,]  0.04578188  1.9610099
[2,] -0.31278019  0.7283852
[3,]  0.15827817 -0.3585138
[1] "sr_ff + sr_rf--------------------------------"
           [,1]       [,2]
[1,] -0.1600604  1.7551677
[2,] -0.7244647  0.3167007
[3,] -0.4592486 -0.9760406


In [16]:
%%R
N_id = 3
sr_termsR = NULL
sr_rfR = NULL
srR = NULL

for (i in 1:N_id) {
  tmp = rep(NA,2)

  # Compute the dot product for the focal and target predictors
  tmp[1] = sum(focal_effects * c(focal_individual_predictors[i]))
  tmp[2] = sum(target_effects * c(target_individual_predictors[i]))
 
  sr_termsR = rbind(sr_termsR, tmp)
  sr_rfR = rbind(sr_rfR, as.vector(t(sr_L %*% sr_raw[i,])))
  # Random effects
  result = sr_sigma*as.vector(t(sr_L %*% sr_raw[i,]))

  srR[[i]] = t(result + tmp)
}
print('sr_ff--------------------------------')
print(sr_termsR)
print('sr_rf --------------------------------')
print(sr_rfR)
print('sr_ff + sr_rf--------------------------------')
print(do.call('rbind',srR))


[1] "sr_ff--------------------------------"
          [,1]       [,2]
tmp -0.2058423 -0.2058423
tmp -0.4116845 -0.4116845
tmp -0.6175268 -0.6175268
[1] "sr_rf --------------------------------"
           [,1]       [,2]
[1,]  0.1878438  1.2028713
[2,] -1.2833426  0.4467870
[3,]  0.6494181 -0.2199102
[1] "sr_ff + sr_rf--------------------------------"
           [,1]       [,2]
[1,] -0.1600604  1.7551677
[2,] -0.7244647  0.3167007
[3,] -0.4592486 -0.9760406


In [17]:
sample = True

focal_effects = dist.normal(0, 1, shape=(1,), sample = sample, name = 'focal_effects')
target_effects =  dist.normal( 0, 1, shape= (1,), sample = sample, name = 'target_effects')
ff = jnp.stack([focal_effects @ focal_individual_predictors, target_effects @  target_individual_predictors], axis = -1)
print("sr_ff--------------------------------")
print(ff)
sr_raw =  dist.normal(0, 1, shape=(2, N_id), name = 'sr_raw', sample = sample)
sr_sigma =  dist.exponential( 1, shape= (2,), name = 'sr_sigma', sample = sample)
sr_L = dist.lkjcholesky(2, 2, name = "sr_L", sample = sample)
rf = numpyro.deterministic('sr_rf',(((sr_L @ sr_raw).T * sr_sigma)))
print("sr_rf--------------------------------")
print(rf)
print("sr_ff + sr_rf--------------------------------")
print(rf + ff)


sr_ff--------------------------------
[[-0.20584226 -0.20584226]
 [-0.4116845  -0.4116845 ]
 [-0.61752677 -0.61752677]]
sr_rf--------------------------------
[[ 0.04578188  1.9610099 ]
 [-0.3127802   0.7283852 ]
 [ 0.15827817 -0.35851383]]
sr_ff + sr_rf--------------------------------
[[-0.16006038  1.7551676 ]
 [-0.7244647   0.3167007 ]
 [-0.4592486  -0.9760406 ]]


## <a id='toc1_4_'></a>[SR to dyadic format](#toc0_)

In [18]:
%%R
sr = do.call('rbind',srR)
stack_sr = matrix(0, 3,3)
for( i in 1:N_id ) {
    for( j in 1:N_id ) {
        if( i != j ) 
            stack_sr[i,j] = sr[i,1] + sr[j,2]
    }
}
stack_sr

         [,1]       [,2]      [,3]
[1,] 0.000000  0.1566403 -1.136101
[2,] 1.030703  0.0000000 -1.700505
[3,] 1.295919 -0.1425479  0.000000


In [19]:
bi.net.node_effects_to_dyadic_format(sr_terms + sr_rf)

Array([[ 0.15664032,  1.0307028 ],
       [-1.136101  ,  1.295919  ],
       [-1.7005053 , -0.1425479 ]], dtype=float32)

# <a id='toc2_'></a>[Dyadic effect](#toc0_)

In [20]:
dyadic_predictorsM = jnp.zeros((3, 3))
dyadic_predictorsM = dyadic_predictorsM .at[0,1].set(1)
dyadic_predictorsM = dyadic_predictorsM .at[0,2].set(1)
dyadic_predictorsM = dyadic_predictorsM .at[1,0].set(1)
dyadic_predictorsM = dyadic_predictorsM .at[2,0].set(1)
print(dyadic_predictorsM)
dyadic_predictors = dyadic_predictorsM
dyadic_predictors = bi.net.prepare_dyadic_effect(dyadic_predictors)
print(dyadic_predictors)

[[0. 1. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[1. 1.]
 [1. 1.]
 [0. 0.]]


## <a id='toc2_1_'></a>[Dyadic fixed effects](#toc0_)

In [21]:
d_terms, dyad_effects = bi.net.dyadic_terms(dyadic_predictors, sample = True, diag = True)

dyad_effects--------------------------------------------------------------------------------
[-0.20584226]
rf--------------------------------------------------------------------------------
[[-0.20584226 -0.20584226]
 [-0.20584226 -0.20584226]
 [-0.         -0.        ]]


## <a id='toc2_2_'></a>[Dyadic random effects](#toc0_)

In [22]:
d_rf, dr_raw, dr_sigma, dr_L = bi.net.dyadic_random_effects(d_terms.shape[0], cholesky_density = 2, sample = True, diag = True)

dr_raw--------------------------------------------------------------------------------
[[ 0.18784384 -1.2833426   0.6494181 ]
 [ 1.2490593   0.24447003 -0.11744965]]
dr_sigma--------------------------------------------------------------------------------
[0.5420704]
dr_L--------------------------------------------------------------------------------
[[ 1.          0.        ]
 [-0.16010584  0.9870998 ]]
rf--------------------------------------------------------------------------------
[[ 0.10182459  0.6520409 ]
 [-0.695662    0.24218999]
 [ 0.35203034 -0.11920679]]


## <a id='toc2_3_'></a>[R equivalent](#toc0_)

In [23]:
dyadic_predictorsM = np.array(dyadic_predictorsM)
d_terms = np.array(d_terms)
dyad_effects = np.array(dyad_effects)
d_rf = np.array(d_rf)
dr_rawM = np.array(bi.net.edgl_to_mat(dr_raw.T, 3))
dr_sigma = np.array(dr_sigma)
dr_L = np.array(dr_L)

In [24]:
%%R -i dyadic_predictorsM
dyad_individual_predictors = dyadic_predictorsM

In [25]:
%%R -i d_terms 
d_terms

           [,1]       [,2]
[1,] -0.2058423 -0.2058423
[2,] -0.2058423 -0.2058423
[3,]  0.0000000  0.0000000


In [26]:
%%R -i dyad_effects
dyad_effects

[1] -0.2058423


In [27]:
%%R -i d_rf
d_rf

           [,1]       [,2]
[1,]  0.1018246  0.6520409
[2,] -0.6956620  0.2421900
[3,]  0.3520303 -0.1192068


In [28]:
%%R -i dr_rawM
dr_rawM

         [,1]       [,2]       [,3]
[1,] 0.000000  0.1878438 -1.2833426
[2,] 1.249059  0.0000000  0.6494181
[3,] 0.244470 -0.1174496  0.0000000


In [29]:
%%R -i dr_sigma
dr_sigma

[1] 0.5420704


In [30]:
%%R -i dr_L
dr_L

           [,1]      [,2]
[1,]  1.0000000 0.0000000
[2,] -0.1601058 0.9870998


In [53]:
%%R
c(dr_rawM[i, j], dr_rawM[j, i])

[1]  0.6494181 -0.1174496


In [54]:
%%R
N_id = 3
dr = matrix(0, nrow = 3, ncol = 3)
dr_scrap = NULL
dr_rf = NULL
dr_ff = NULL
for (i in 1:(N_id - 1)) {
  for (j in (i + 1):N_id) {
    scrap <- c(dr_rawM[i, j], dr_rawM[j, i])
    dr_scrap = rbind(dr_scrap, scrap)

    
    dr_rf = rbind(dr_rf, t(rep(dr_sigma, 2) * (dr_L %*% scrap)))
    scrap <- rep(dr_sigma, 2) * (dr_L %*% scrap)

    dr_ff = rbind(dr_ff, sum(dyad_effects %*% as.vector(dyad_individual_predictors[i, j])))

    dr[i, j] <- scrap[1] + sum(dyad_effects %*% as.vector(dyad_individual_predictors[i, j]))
    dr[j, i] <- scrap[2] + sum(dyad_effects %*%  as.vector(dyad_individual_predictors[j, i]))
  }
}
print('dr_ff------------------------------------------------')
print(dr_ff)
print('dr_scrap------------------------------------------------')
print(dr_scrap)
print('dr_rf------------------------------------------------')
print(dr_rf)
print('dr_scrap + dr_rf------------------------------------------------')
print(dr)


[1] "dr_ff------------------------------------------------"
           [,1]
[1,] -0.2058423
[2,] -0.2058423
[3,]  0.0000000
[1] "dr_scrap------------------------------------------------"
            [,1]       [,2]
scrap  0.1878438  1.2490593
scrap -1.2833426  0.2444700
scrap  0.6494181 -0.1174496
[1] "dr_rf------------------------------------------------"
           [,1]       [,2]
[1,]  0.1018246  0.6520409
[2,] -0.6956620  0.2421900
[3,]  0.3520303 -0.1192068
[1] "dr_scrap + dr_rf------------------------------------------------"
           [,1]       [,2]       [,3]
[1,] 0.00000000 -0.1040177 -0.9015043
[2,] 0.44619868

  0.0000000  0.3520303
[3,] 0.03634773 -0.1192068  0.0000000


In [55]:
dr_rf = (((dr_L @ dr_raw).T * jnp.repeat(dr_sigma, 2)))
dr_ff = dyad_effects * dyadic_predictors
print('dr_ff------------------------------------------------')
print(dr_ff)
print('dr_scrap------------------------------------------------')
print(dr_raw.T)
print('dr_rf------------------------------------------------')
print(dr_rf)
print('dr_scrap + dr_rf------------------------------------------------')
print(dr_ff + dr_rf)

dr_ff------------------------------------------------
[[-0.20584226 -0.20584226]
 [-0.20584226 -0.20584226]
 [-0.         -0.        ]]
dr_scrap------------------------------------------------
[[ 0.18784384  1.2490593 ]
 [-1.2833426   0.24447003]
 [ 0.6494181  -0.11744965]]
dr_rf------------------------------------------------
[[ 0.10182459  0.6520409 ]
 [-0.695662    0.24218999]
 [ 0.35203034 -0.11920679]]
dr_scrap + dr_rf------------------------------------------------
[[-0.10401767  0.44619864]
 [-0.9015043   0.03634773]
 [ 0.35203034 -0.11920679]]


# Block model

In [63]:
bi.logit(0.1/jnp.sqrt(3))

Array(-2.7924225, dtype=float32, weak_type=True)

In [62]:
%%R
p = 0.1 / sqrt(3)
log(p / (1 -p))

[1] -2.792422


### R data

In [99]:
%%R
# Make data
set.seed(1)
N_id = 10
V = 3
groups_1 = rep("Any",N_id)                                                                                  # Block Variable 1, Intercept
groups_2 = sample( c("Red","White","Blue") , size=N_id , replace=TRUE , prob=c(0.5, 0.25, 0.25) )           # Block Variable 2, Color
groups_3 = sample( c("Strangeness", "Charm") , size=N_id , replace=TRUE , prob=c(0.5,0.5) )                 # Block Variable 3, Flavor 

groups = data.frame(Intercept=as.numeric(factor(groups_1)), Color=as.numeric(factor(groups_2)), Flavor=as.numeric(factor(groups_3)))

# Block parameter structure
B_1 = matrix(-10,nrow=1,ncol=1)
B_2 = matrix(rnorm(9,0,3),nrow=3,ncol=3)
B_3 = matrix(rnorm(4,0,3),nrow=2,ncol=2)

diag(B_2) = diag(B_2) + 2
diag(B_3) = diag(B_3) + 3.5

B = list(B_1, B_2, B_3)

# Build offset matrix
block_offsets = matrix(NA, N_id, N_id)

# Loop over upper triangle and create ties from i to j, and j to i
for ( i in 1:(N_id-1) ){
    for ( j in (i+1):N_id){

 # Loop over block variables
   B_i_j = B_j_i = c()
  for(v in 1:V){
    B_i_j[v] =  B[[v]][groups[i,v] , groups[j,v] ]
    B_j_i[v] =  B[[v]][groups[j,v] , groups[i,v] ]
  }

 block_offsets[i,j] =  sum(B_i_j)
 block_offsets[j,i] =  sum(B_j_i)

 }}

# Now reshape into N_dydads X 2 matrix
melted = cbind(block_offsets[lower.tri(block_offsets,diag=FALSE)],
               block_offsets[upper.tri(block_offsets,diag=FALSE)])

block_offsets
melted

# quick check is the same both column 1 and column 2 becuase ind_1 and ind_2 are in the same block for both color and flavor
melted[1,]

[1] 0.6564967 0.6564967


In [98]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import rpy2py
import pandas as pd
pandas2ri.activate()
grp = ro.globalenv['groups']
grp = pandas2ri.rpy2py(grp)
grp_1 = jnp.array(grp['Intercept'].values).astype('int32') - 1
grp_2 = jnp.array(grp['Color'].values).astype('int32')  - 1
grp_3 = jnp.array(grp['Flavor'].values).astype('int32')  - 1

B = ro.globalenv['B']
B1 = B[0]
B2 = B[1]
B3 = B[2]

B1_edgl = Net.block_prior_to_edglelist(grp_1, B1)
B2_edgl = Net.block_prior_to_edglelist(grp_2, B2)
B3_edgl = Net.block_prior_to_edglelist(grp_3, B3)
res = B1_edgl + B2_edgl + B3_edgl
res[0,:]


Array([0.6564965, 0.6564965], dtype=float32)